#### Funcitons (IGNORE)

In [65]:
# import packages that will be used for analysis
import random
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

##### Get Stock Data

In [66]:
import yfinance as yf
missing_data_tickers = [] # use this as a list of tickers with missing data

def get_data_from_start_to_end(ticker, start_date, end_date):
    global missing_data_tickers  # Use the global list to accumulate missing tickers
    try:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        if stock_data.empty:
            missing_data_tickers.append(ticker)
            raise ValueError(f"Stock data for ticker {ticker} during the period from {start_date} to {end_date} was not found.")
        return stock_data
    except Exception as e:
        print(f"An error occurred for ticker {ticker}: {e}")
        missing_data_tickers.append(ticker)
        return None


In [67]:
# for a variety of periods load in different list of tickers
def download_stock_data_for_periods(tickers, periods):
    all_data = {}
    
    for period, (start_date, end_date) in periods.items():
        period_data = {}
        for ticker in tickers:
            data = get_data_from_start_to_end(ticker, start_date, end_date)
            if data is not None:
                period_data[ticker] = data
        all_data[period] = period_data
    
    return all_data

In [68]:
import pandas as pd

# Get the adjusted close prices
adj_close_sector_etf = {}

# Create adjusted close price only listing of sector ETFs
def get_adjusted_closed_price(nested_dict, tickers, periods):
    for period in periods:
        stock_price_df = pd.DataFrame()  # Create a new DataFrame for each period
        for ticker in tickers:
            stock_price_df[ticker] = nested_dict[period][ticker]['Adj Close']
        
        adj_close_sector_etf[period] = stock_price_df  # Store the complete DataFrame for the period
    
    return adj_close_sector_etf

##### Investment Portfolio

In [69]:
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)
    
from datetime import timedelta
def individual_stock(bb_signals_nd, adj_close_nd, periods_date, periods_list, tickers, initial_investment, percent_to_buy, percent_to_sell):
    # Portfolio summary - nested dictionary for each period and ticker
    portfolio_summary = {period: {ticker: pd.DataFrame() for ticker in tickers} for period in periods_list}

    # Set data to be accessed
    adj_close_data = adj_close_nd
    bollinger_band_data = bb_signals_nd

    all_data = {
        'Stock Tracker': portfolio_summary,
        'Adjusted Close Price': adj_close_nd,
        'Bollinger Band Signal': bollinger_band_data
    }

    # Loop through each economic period
    for period in periods_list:
        # Create the date range for the current period
        date_range = pd.date_range(start=pd.to_datetime(periods_date[period][0]), end=pd.to_datetime(periods_date[period][1]) - timedelta(days=120))
        # Get random dates for stochastic modeling
        start_dates = np.random.choice(date_range, size=1, replace=False)

        # Loop through sampled start dates
        for start_date in start_dates:
            time_stamp = pd.to_datetime(start_date)

            # Initialize account balance and shares for each ticker
            account_balance = {ticker: initial_investment for ticker in tickers}
            shares_number = {ticker: 0 for ticker in tickers}  # Initialize share count for each ticker

            # Extract the adjusted close and signal data for the time period
            adj_close_period = adj_close_data[period].loc[time_stamp:time_stamp + timedelta(days=120)]
            bb_signals_period = bollinger_band_data[period].loc[time_stamp:time_stamp + timedelta(days=120)]

            # Iterate over each row in the Bollinger Band signals (day by day)
            for row_idx, row in bb_signals_period.iterrows():

                # Tracking for each ticker individually
                for col_idx, signal in enumerate(row):
                    ticker = tickers[col_idx]  # Get ticker for each column
                    adj_close_price = adj_close_period.loc[row_idx, ticker]  # Get adjusted close price for that ticker

                    # Initialize stock tracker for current ticker
                    stock_tracker = all_data['Stock Tracker'][period][ticker]

                    # Handle Buy action
                    if signal == 'Buy':
                        amount_to_buy = percent_to_buy * account_balance[ticker]
                        if account_balance[ticker] >= amount_to_buy:
                            # Calculate shares to buy
                            shares_to_buy = amount_to_buy / adj_close_price
                            shares_number[ticker] += shares_to_buy

                            # Track investment for the current period
                            stock_tracker = stock_tracker.append({
                                'Date': row_idx,
                                'Share Price': adj_close_price,
                                'Signal': 'Buy',
                                'Buy/Sell Amount ($)': amount_to_buy,
                                'Buy/Sell Number of Shares': shares_to_buy,
                                'Shares ($) Ownership': shares_number[ticker] * adj_close_price,  # Update based on current price
                                'Shares Ownership': shares_number[ticker],
                                'Account Balance': account_balance[ticker] - amount_to_buy,  # Update balance after buying
                                'Stock Value ($)': shares_number[ticker] * adj_close_price,  # Value of the stock owned
                                'Total Value ($)': (account_balance[ticker] - amount_to_buy) + (shares_number[ticker] * adj_close_price),  # Total value of investment (balance + stock value)
                                'Profit ($)': ((account_balance[ticker] - amount_to_buy) + (shares_number[ticker] * adj_close_price)) - initial_investment  # Profit calculation
                            }, ignore_index=True)

                            # Update account balance after buying
                            account_balance[ticker] -= amount_to_buy

                    # Handle Sell action
                    elif signal == 'Sell':
                        if shares_number[ticker] > 0:  # Ensure we have shares to sell
                            amount_to_sell = percent_to_sell * (shares_number[ticker] * adj_close_price)
                            shares_to_sell = amount_to_sell / adj_close_price
                            if shares_number[ticker] >= shares_to_sell:
                                shares_number[ticker] -= shares_to_sell

                                # Track the sell action
                                stock_tracker = stock_tracker.append({
                                    'Date': row_idx,
                                    'Share Price': adj_close_price,
                                    'Signal': 'Sell',
                                    'Buy/Sell Amount ($)': amount_to_sell,
                                    'Buy/Sell Number of Shares': shares_to_sell,
                                    'Shares ($) Ownership': shares_number[ticker] * adj_close_price,  # Update based on current price
                                    'Shares Ownership': shares_number[ticker],
                                    'Account Balance': account_balance[ticker] + amount_to_sell,  # Update balance after selling
                                    'Stock Value ($)': shares_number[ticker] * adj_close_price,  # Value of the stock owned
                                    'Total Value ($)': (account_balance[ticker] + amount_to_sell) + (shares_number[ticker] * adj_close_price),  # Total value of investment (balance + stock value)
                                    'Profit ($)': ((account_balance[ticker] + amount_to_sell) + (shares_number[ticker] * adj_close_price)) - initial_investment  # Profit calculation
                                }, ignore_index=True)

                                # Update account balance after selling
                                account_balance[ticker] += amount_to_sell

                    # Handle Hold action (no action taken)
                    else:
                        # Track the hold state
                        stock_tracker = stock_tracker.append({
                            'Date': row_idx,
                            'Share Price': adj_close_price,
                            'Signal': 'Hold',
                            'Buy/Sell Amount ($)': 0,
                            'Buy/Sell Number of Shares': 0,
                            'Shares ($) Ownership': shares_number[ticker] * adj_close_price,  # Update based on current price
                            'Shares Ownership': shares_number[ticker],
                            'Account Balance': account_balance[ticker],  # No change in balance
                            'Stock Value ($)': shares_number[ticker] * adj_close_price,  # Value of the stock owned
                            'Total Value ($)': account_balance[ticker] + (shares_number[ticker] * adj_close_price),  # Total value of investment (balance + stock value)
                            'Profit ($)': (account_balance[ticker] + (shares_number[ticker] * adj_close_price)) - initial_investment  # Profit calculation
                        }, ignore_index=True)

                    # Save the updated tracker back to portfolio summary
                    all_data['Stock Tracker'][period][ticker] = stock_tracker.copy()

    # Return the complete portfolio summary for all periods and tickers
    return all_data

##### MACD Signal

In [70]:
def macd_components(nested_dict,periods,tickers):
    for period in periods:
        for ticker in tickers:
            # get the short and long ema
            nested_dict[period][ticker]['short_ema'] = nested_dict[period][ticker]['Close'].ewm(span=12, adjust=False).mean()
            nested_dict[period][ticker]['long_ema'] = nested_dict[period][ticker]['Close'].ewm(span=26, adjust=False).mean()

            # create the MACD line
            nested_dict[period][ticker]['macd_line'] = nested_dict[period][ticker]['short_ema'] - nested_dict[period][ticker]['long_ema']
            nested_dict[period][ticker]['signal_line'] = nested_dict[period][ticker]['macd_line'].ewm(span=9, adjust=False).mean()

In [71]:
def add_macd_signals(nested_dict,periods, tickers, short_window=12, long_window=26, signal_window=9):
    """
    Function to calculate MACD, Signal Line, and generate Buy/Sell/Hold signals based on crossovers.
    """
    for period in periods:
        for ticker in tickers:

            data = nested_dict[period][ticker]

            # Generate 'Buy', 'Sell', 'Hold' signals using np.where() based on crossovers
            data['Signal'] = np.where(data['macd_line'] > data['signal_line'], 'Buy', 
                                    np.where(data['macd_line'] < data['signal_line'], 'Sell', 'Hold'))

In [72]:
def generate_signals(nested_dict, periods, tickers):
    for period in periods:
        for ticker in tickers:
            # Create lists to store buy and sell signals
            buy_signals = []
            sell_signals = []

            # Initialize variables to track the last state (to detect crossovers)
            last_macd_above_signal = None

            # Loop through each row in the MACD dataframe
            for i in range(len(nested_dict[period][ticker]['macd_line'])):
                macd_line = nested_dict[period][ticker]['macd_line'].iloc[i]
                signal_line = nested_dict[period][ticker]['signal_line'].iloc[i]

                # Determine buy/sell signal based on MACD crossover
                if macd_line > signal_line and (last_macd_above_signal is False or last_macd_above_signal is None):
                    buy_signals.append(nested_dict[period][ticker]['Close'].iloc[i])
                    sell_signals.append(None)
                    last_macd_above_signal = True
                elif macd_line < signal_line and (last_macd_above_signal is True or last_macd_above_signal is None):
                    sell_signals.append(nested_dict[period][ticker]['Close'].iloc[i])
                    buy_signals.append(None)
                    last_macd_above_signal = False
                else:
                    # No signal
                    buy_signals.append(None)
                    sell_signals.append(None)

            # Store buy and sell signals in the nested dictionary
            nested_dict[period][ticker]['buy_signals'] = buy_signals
            nested_dict[period][ticker]['sell_signals'] = sell_signals



In [73]:
def collect_signals(nested_dict, periods, tickers):
    # Initialize an empty dictionary to hold DataFrames for each period
    bb_nested_dict = {}

    for period in periods:
        # Create a DataFrame for each period with the tickers as columns
        signals_period = pd.DataFrame(columns=tickers)
        
        # Loop through each ticker and extract the 'Signal'
        for ticker in tickers:
            signals_period[ticker] = nested_dict[period][ticker]['Signal']
        
        # Store the DataFrame in the dictionary using the period as the key
        bb_nested_dict[period] = signals_period

    # Return the dictionary containing DataFrames for each period
    return bb_nested_dict

# Moving Average Convergence Divergence (MACD)
The MACD is another popular technical analysis technique, it concentrates on the momentum of a stock. It has two lines which oscillate between one another. The first line, referred to as the MACD line is found by looking at longer term EMA (typically 26) / shorter term EMA (typically 12). The second line is referred to as the signal line which is typically a 9 day EMA of the MACD line. It can suggest the following:

- Bullish Signal: When the MACD Line crosses above the Signal Line, it can indicate upward momentum, a possible buy signal.
- Bearish Signal: When the MACD Line crosses below the Signal Line, it can indicate downward momentum, a possible sell signal.
- Divergence: Price action diverging from the MACD can signal potential reversals.

In [74]:
# create time periods for where this takes place
economic_cycle_periods = {

    "trough": ("2008-10-01", "2009-06-01"),
    "expansion": ("2012-01-01", "2015-01-01"),
    "peak": ("2019-06-01", "2020-02-01"),
    "contraction": ("2007-12-01", "2008-10-01"),
    'all_data': ('2005-01-01','2024-06-01')
}

economic_cycle_periods_list = ['trough','expansion','peak','contraction','all_data']

In [75]:
# create etf tickers for sectors
sector_etf_tickers = [
    'XLB', # materials sector
    'XLI', # industrials sector
    'XLF', # financials
    'XLK', # information technology
    'XLY', # consumer discretionary
    'XLP', # consumer staples
    'XLE', # energy
    'XLV', # healthcare
    'VOX', # communication services
    'XLU', # utilities
    'IYR' # real estate
    ]

In [76]:
# save nested dictionary data as a variable to be accessed.
sector_etf_data = download_stock_data_for_periods(sector_etf_tickers,economic_cycle_periods)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [77]:
macd_components(sector_etf_data,economic_cycle_periods_list,sector_etf_tickers)

In [78]:
add_macd_signals(sector_etf_data,economic_cycle_periods_list,sector_etf_tickers)

In [79]:
sector_etf_data['expansion']['XLV']['Signal'].value_counts()

Buy     386
Sell    367
Hold      1
Name: Signal, dtype: int64

In [80]:
macd_signals = collect_signals(sector_etf_data,economic_cycle_periods_list,sector_etf_tickers)

In [85]:
for period in economic_cycle_periods_list:
    macd_signals[period].to_csv(f'/Users/ben_nicholson/Visual_Code_Projects/Personal_Projects/Financial Capstone Project/data/signal_data/macd_signals/{period}_macd_signal.csv')

In [81]:
adj_close_sector_etf = get_adjusted_closed_price(sector_etf_data,sector_etf_tickers,economic_cycle_periods_list)

In [82]:
macd_investment = individual_stock(macd_signals,adj_close_sector_etf,economic_cycle_periods,economic_cycle_periods_list,sector_etf_tickers,100,0.05,0.05)

In [83]:
macd_investment['Stock Tracker']['expansion']['XLF']

,Date,Share Price,Signal,Buy/Sell Amount ($),Buy/Sell Number of Shares,Shares ($) Ownership,Shares Ownership,Account Balance,Stock Value ($),Total Value ($),Profit ($)
0,2012-02-09,9.412790,Buy,5.000000,0.531192,5.000000,0.531192,95.000000,5.000000,100.000000,0.000000
1,2012-02-10,9.323206,Buy,4.750000,0.509481,9.702413,1.040674,90.250000,9.702413,99.952413,-0.047587
2,2012-02-13,9.406391,Buy,4.512500,0.479727,14.301482,1.520401,85.737500,14.301482,100.038982,0.038982
3,2012-02-14,9.316805,Buy,4.286875,0.460123,18.452150,1.980523,81.450625,18.452150,99.902775,-0.097225
4,2012-02-15,9.259218,Sell,0.916905,0.099026,17.421193,1.881497,82.367530,17.421193,99.788723,-0.211277
...,...,...,...,...,...,...,...,...,...,...,...
79,2012-06-04,8.576236,Sell,0.523022,0.060985,9.937422,1.158716,86.811638,9.937422,96.749060,-3.250940
80,2012-06-05,8.717463,Sell,0.505053,0.057936,9.596010,1.100780,87.316691,9.596010,96.912701,-3.087299
81,2012-06-06,8.974233,Sell,0.493933,0.055039,9.384725,1.045741,87.810624,9.384725,97.195349,-2.804651
82,2012-06-07,8.967814,Buy,4.390531,0.489588,13.768544,1.535329,83.420093,13.768544,97.188637,-2.811363
